In [1]:
from tensorflow.keras.layers import Conv2DTranspose, Concatenate, UpSampling2D
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import MaxPooling2D, Input
from tensorflow.keras import Model
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.applications import ResNet50V2


def convolution_block(inputs, filters, kernel_size=3, activation='relu'):
    """UNET convolution block containing Conv2D -> BatchNorm -> Activation

    # Arguments
        inputs: Keras/tensorflow tensor input.
        filters: Int. Number of filters.
        kernel_size: Int. Kernel size of convolutions.
        activation: String. Activation used convolution.

    # Returns
        Keras/tensorflow tensor.
    """
    kwargs = {'use_bias': False, 'kernel_initializer': 'he_uniform'}
    x = Conv2D(filters, kernel_size, (1, 1), 'same', **kwargs)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    return x


def upsample_block(x, filters, branch):
    """UNET upsample block. This block upsamples ``x``, concatenates a
    ``branch`` tensor and applies two convolution blocks:
    Upsample -> Concatenate -> 2 x ConvBlock.

    # Arguments
        x: Keras/tensorflow tensor.
        filters: Int. Number of filters
        branch: Tensor to be concatated to the upsamples ``x`` tensor.

    # Returns
        A Keras tensor.
    """
    x = UpSampling2D(size=2)(x)
    x = Concatenate(axis=3)([x, branch])
    x = convolution_block(x, filters)
    x = convolution_block(x, filters)
    return x


# def transpose_block(x, filters, branch):
#     """UNET transpose block. This block upsamples ``x``, concatenates a
#     ``branch`` tensor and applies two convolution blocks:
#     Conv2DTranspose -> Concatenate -> 2 x ConvBlock.

#     # Arguments
#         x: Keras/tensorflow tensor.
#         filters: Int. Number of filters
#         branch: Tensor to be concatated to the upsamples ``x`` tensor.

#     # Returns
#         A Keras tensor.
#     """
#     x = Conv2DTranspose(filters, 4, (2, 2), 'same', use_bias=False)(x)
#     x = BatchNormalization()(x)
#     x = Activation('relu')(x)
#     x = Concatenate(axis=3)([x, branch])
#     x = convolution_block(x, filters)
#     return x


def freeze_model(model):
    """Freezes gradient pass for the entire model

    # Arguments:
        model: Keras/tensorflow model

    # Returns:
        A Keras/tensorflow model
    """
    for layer in model.layers:
        layer.trainable = False
    return model


def get_tensors(model, layer_names):
    """Gets all the tensor outputs of the given layer names.

    # Arguments
        model: Keras/tensorflow model.
        layer_names: List of strings which each string is a layer name.

    # Returns
        List of Keras tensors.
    """
    tensors = []
    for layer_name in layer_names:
        tensors.append(model.get_layer(layer_name).output)
    return model, tensors


def build_backbone(BACKBONE, shape, branch_names, weights,
                   frozen=False, input_tensor=None):
    """Builds ``BACKBONE`` class for UNET model.

    # Arguments
        BACKBONE: Class for instantiating a backbone model
        shape: List of integers: ``(H, W, num_channels)``.
        branch_names: List of strings containing layer names of ``BACKBONE()``.
        weights: String or ``None``.
        frozen: Boolean. If True ``BACKBONE()`` updates are frozen.
        input_tensor: Input tensor. If given ``shape`` is overwritten and this
            tensor is used instead as input.

    # Returns
    """
    kwargs = {'include_top': False, 'input_shape': shape, 'weights': weights}
    if input_tensor is not None:
        kwargs.pop('input_shape')
        kwargs['input_tensor'] = input_tensor
    backbone = BACKBONE(**kwargs)

    if frozen:
        backbone = freeze_model(backbone)

    backbone, branch_tensors = get_tensors(backbone, branch_names)
    return backbone, branch_tensors


def build_UNET(num_classes, backbone, branch_tensors,
               decoder, decoder_filters, activation, name):
    """Build UNET with a given ``backbone`` model.

    # Arguments
        num_classes: Integer used for output number of channels.
        backbone: Instantiated backbone model.
        branch_tensors: List of tensors from ``backbone`` model
        decoder: Function used for upsampling and decoding the output.
        decoder_filters: List of integers used in each application of decoder.
        activation: Output activation of the model.
        name: String. indicating the name of the model.

    # Returns
        A UNET Keras/tensorflow model.
    """
    inputs, x = backbone.input, backbone.output
    if isinstance(backbone.layers[-1], MaxPooling2D):
        x = convolution_block(x, 512)
        x = convolution_block(x, 512)

    for branch, filters in zip(branch_tensors, decoder_filters):
        x = decoder(x, filters, branch)

    kwargs = {'use_bias': True, 'kernel_initializer': 'glorot_uniform'}
    x = Conv2D(num_classes, 3, (1, 1), 'same', **kwargs)(x)
    outputs = Activation(activation, name='masks')(x)
    model = Model(inputs, outputs, name=name)
    return model


def UNET(input_shape, num_classes, branch_names, BACKBONE, weights,
         freeze_backbone=False, activation='sigmoid', decoder_type='upsample',
         decoder_filters=[256, 128, 64, 32, 16], input_tensor=None,
         name='UNET'):
    """Build a generic UNET model with a given ``BACKBONE`` class.

    # Arguments
        input_shape: List of integers: ``(H, W, num_channels)``.
        num_classes: Integer used for output number of channels.
        branch_names: List of strings containing layer names of ``BACKBONE()``.
        BACKBONE: Class for instantiating a backbone model
        weights: String indicating backbone weights e.g.
            ''imagenet'', ``None``.
        freeze_backbone: Boolean. If True ``BACKBONE()`` updates are frozen.
        decoder_type: String indicating decoding function e.g.
            ''upsample ''transpose''.
        decoder_filters: List of integers used in each application of decoder.
        activation: Output activation of the model.
        input_tensor: Input tensor. If given ``shape`` is overwritten and this
            tensor is used instead as input.
        name: String. indicating the name of the model.

    # Returns
        A UNET Keras/tensorflow model.
    """
    args = [BACKBONE, input_shape, branch_names,
            weights, freeze_backbone, input_tensor]
    backbone, branch_tensors = build_backbone(*args)
    if decoder_type == 'upsample':
        decoder = upsample_block
    if decoder_type == 'transpose':
        decoder = transpose_block

    model = build_UNET(num_classes, backbone, branch_tensors, decoder,
                       decoder_filters, activation, name)
    return model


# def UNET_VGG16(num_classes=1, input_shape=(224, 224, 3), weights='imagenet',
#                freeze_backbone=False, activation='sigmoid',
#                decoder_type='upsample',
#                decode_filters=[256, 128, 64, 32, 16]):
#     """Build a UNET model with a ``VGG16`` backbone.

#     # Arguments
#         input_shape: List of integers: ``(H, W, num_channels)``.
#         num_classes: Integer used for output number of channels.
#         branch_names: List of strings containing layer names of ``BACKBONE()``.
#         BACKBONE: Class for instantiating a backbone model
#         weights: String indicating backbone weights e.g.
#             ''imagenet'', ``None``.
#         freeze_backbone: Boolean. If True ``BACKBONE()`` updates are frozen.
#         decoder_type: String indicating decoding function e.g.
#             ''upsample ''transpose''.
#         decoder_filters: List of integers used in each application of decoder.
#         activation: Output activation of the model.
#         input_tensor: Input tensor. If given ``shape`` is overwritten and this
#             tensor is used instead as input.
#         name: String. indicating the name of the model.

#     # Returns
#         A UNET-VGG16 Keras/tensorflow model.
#     """
#     VGG16_branches = ['block5_conv3', 'block4_conv3', 'block3_conv3',
#                       'block2_conv2', 'block1_conv2']
#     return UNET(input_shape, num_classes, VGG16_branches, VGG16, weights,
#                 freeze_backbone, activation, decoder_type, decode_filters,
#                 name='UNET-VGG16')


# def UNET_VGG19(num_classes=1, input_shape=(224, 224, 3), weights='imagenet',
#                freeze_backbone=False, activation='sigmoid',
#                decoder_type='upsample',
#                decode_filters=[256, 128, 64, 32, 16]):
#     """Build a UNET model with a ``VGG19`` backbone.

#     # Arguments
#         input_shape: List of integers: ``(H, W, num_channels)``.
#         num_classes: Integer used for output number of channels.
#         branch_names: List of strings containing layer names of ``BACKBONE()``.
#         BACKBONE: Class for instantiating a backbone model
#         weights: String indicating backbone weights e.g.
#             ''imagenet'', ``None``.
#         freeze_backbone: Boolean. If True ``BACKBONE()`` updates are frozen.
#         decoder_type: String indicating decoding function e.g.
#             ''upsample ''transpose''.
#         decoder_filters: List of integers used in each application of decoder.
#         activation: Output activation of the model.
#         input_tensor: Input tensor. If given ``shape`` is overwritten and this
#             tensor is used instead as input.
#         name: String. indicating the name of the model.

#     # Returns
#         A UNET-VGG19 Keras/tensorflow model.
#     """

    # VGG19_branches = ['block5_conv4', 'block4_conv4', 'block3_conv4',
    #                   'block2_conv2', 'block1_conv2']
    # return UNET(input_shape, num_classes, VGG19_branches, VGG19, weights,
    #             freeze_backbone, activation, decoder_type, decode_filters,
    #             name='UNET-VGG19')


def UNET_RESNET50(num_classes=1, input_shape=(224, 224, 3), weights='imagenet',
                  freeze_backbone=False, activation='sigmoid',
                  decoder_type='upsample',
                  decode_filters=[256, 128, 64, 32, 16]):
    """Build a UNET model with a ``RESNET50V2`` backbone.

    # Arguments
        input_shape: List of integers: ``(H, W, num_channels)``.
        num_classes: Integer used for output number of channels.
        branch_names: List of strings containing layer names of ``BACKBONE()``.
        BACKBONE: Class for instantiating a backbone model
        weights: String indicating backbone weights e.g.
            ''imagenet'', ``None``.
        freeze_backbone: Boolean. If True ``BACKBONE()`` updates are frozen.
        decoder_type: String indicating decoding function e.g.
            ''upsample ''transpose''.
        decoder_filters: List of integers used in each application of decoder.
        activation: Output activation of the model.
        input_tensor: Input tensor. If given ``shape`` is overwritten and this
            tensor is used instead as input.
        name: String. indicating the name of the model.

    # Returns
        A UNET-RESNET50V2 Keras/tensorflow model.
    """
    RESNET50_branches = ['conv4_block6_1_relu', 'conv3_block4_1_relu',
                         'conv2_block3_1_relu', 'conv1_conv', 'input_resnet50']
    input_tensor = Input(input_shape, name='input_resnet50')
    return UNET(input_shape, num_classes, RESNET50_branches, ResNet50V2,
                weights, freeze_backbone, activation, decoder_type,
                decode_filters, input_tensor, 'UNET-RESNET50')

In [3]:
unet_3 = UNET_RESNET50(input_shape=(224, 224, 3), weights='imagenet')

In [5]:
unet_3.summary()

Model: "UNET-RESNET50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_resnet50 (InputLayer)    [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_resnet50[0][0]']         
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                      

In [6]:
unet_4 = UNET_RESNET50(input_shape=(224, 224, 4), weights=None)
unet_4.summary()

Model: "UNET-RESNET50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_resnet50 (InputLayer)    [(None, 224, 224, 4  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 4)  0           ['input_resnet50[0][0]']         
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  12608       ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                      

In [7]:
import tensorflow as tf
from keras import backend as K
import tifffile
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import json
import csv
import random
from time import time
import shutil

# Definition des Data-Generators
class CustomDataGenerator(tf.keras.utils.Sequence):
 
    def __init__(self, batch_size, img_directory, msk_directory, shuffle= False, augment= False):
        self.batch_size = batch_size
        self.img_directory = img_directory
        self.msk_directory = msk_directory
        self.list_img_IDs = os.listdir(self.img_directory)
        self.list_msk_IDs = os.listdir(self.msk_directory)
        self.shuffle = shuffle
        self.augment = augment

    def augment_data(self, x, y):     
        x_flip = x
        y_flip = y

        # zufällige horizontale & vertikale Flips
        horiz = random.randint(0, 9)
        if horiz <= 4:
            x_flip = np.fliplr(x)
            y_flip = np.fliplr(y)

        vert = random.randint(0, 9)
        if vert <= 4:
            x_flip = np.flipud(x_flip)
            y_flip = np.flipud(y_flip)
        
        return x_flip, y_flip

    def __len__(self):

        return len(os.listdir(self.img_directory)) // self.batch_size
    
    def __getitem__(self, index):
        batch_img_IDs = self.list_img_IDs[index*self.batch_size : (index+1)*self.batch_size]
        batch_msk_IDs = self.list_msk_IDs[index*self.batch_size : (index+1)*self.batch_size]

        images = []
        masks = []
        for img_id, msk_id in zip(batch_img_IDs, batch_msk_IDs):
            # einlesen Bild
            img_path = os.path.join(self.img_directory, img_id)
            with open(img_path, 'rb') as f:
                image = tifffile.imread(f)

            # Transformation in "channels_last"-Format
            image = np.moveaxis(image, 0, -1)
            
            # einlesen Maske
            msk_path = os.path.join(self.msk_directory, msk_id)
            with open(msk_path, 'rb') as f:
                mask = tifffile.imread(f)

            # Erstellen einer zusätzlichen Achse um Tensor-Dimension zu erreichen
            mask = mask[:, :, np.newaxis]

            # Data Augmentation
            if self.augment:
                image, mask = self.augment_data(image, mask)

            # Skalierung der Werte
            images.append((image / 127.5) - 1)
            masks.append(mask/255)
        
        return (np.array(images), np.array(masks))
    
    def on_epoch_end(self):
        if self.shuffle:
            a = self.list_img_IDs
            b = self.list_msk_IDs

            c = list(zip(a, b))

            random.shuffle(c)

            self.list_img_IDs, self.list_msk_IDs = zip(*c)


# Definition des Dice-Koeffizienten
def Dice_coefficient(y_true, y_pred, smooth=10e-6):        
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return dice


# Ableitung einer zu minimierenden Loss-Funktion aus Dice-Koeffzient
def Dice_loss(y_true, y_pred):
    return 1 - Dice_coefficient(y_true, y_pred)


# Rückgängig machen der Normalisierung zur korrekten Anzeige der Bilder
def reverse_scaling(image):
    return (((image + 1) / 2 )* 255).astype(np.uint8)


def load_model(model_type):
    # Speicherpfade der verschiedenen Architekturen
    model_dict = {
        'BN': './model_config_files/conf.json',
        'CONV': './model_config_files/conf_RGB_addConv3.json',
        'SPLIT': './model_config_files/conf_splitRGB.json'
        }

    # Auswahl der Architektur entsprechend der verwendeten Variante
    path = model_dict[model_type]

    # Laden der JSON
    with open(path, 'r', encoding='utf-8') as f:
        new_conf = json.load(f)

    # Laden des Modells aus JSON
    unet = tf.keras.Model().from_config(new_conf)

    # Wo die shape der Gewichte des Layers es zulässt, werden immer die selben zufällig initialisierten Gewichte verwendet
    random_path = './saved_weights/unet_resnet50v2_random.npy'

    # entsprechend der Variante müssen Gewichte unterschiedlich gesetzt werden
    if model_type == 'BN':
        loaded_weights = np.load(random_path, allow_pickle= True)
        unet.set_weights(loaded_weights)


    elif model_type == 'CONV':
        # zufällige Gewichte des neu erstellten U-Nets
        unet_weights = unet.get_weights()

        # gespeicherte zufällige Gewichte für den einheitlichen Decoder-Teil des U-Nets
        loaded_weights = np.load(random_path, allow_pickle= True)

        # Leere Liste für neue Gewichte
        updated_weights = []

        # kernel und bias Gewichte des zusätzlichen Convolution-layer
        for i in range(2):
            updated_weights.append(unet_weights[i])

        # einfügen aller weiteren Gewichte
        for unet_w, loaded_w in zip(unet_weights[2:], loaded_weights):
            # für den 2. Convolution-layer passt die shape nicht, bleibt daher unberührt
            if unet_w.shape != loaded_w.shape:
                updated_weights.append(unet_w)

            # alle anderen werden durch die geladenen ersetzt
            else:
                updated_weights.append(loaded_w)

        # Einsetzen der aktualisierten Gewichte
        unet.set_weights(updated_weights)

    elif model_type == 'SPLIT':
        # zufällige Gewichte des neu erstellten U-Nets
        unet_weights = unet.get_weights()

        # gespeicherte zufällige Gewichte für den einheitlichen Decoder-Teil des U-Nets
        loaded_weights = np.load(random_path, allow_pickle= True)

        # Leere Liste für neue Gewichte
        updated_weights = []

        # kernel und bias Gewichte des zusätzlichen Convolution-layer
        for i in range(4):
            updated_weights.append(unet_weights[i])

        # einfügen aller weiteren Gewichte
        for unet_w, loaded_w in zip(unet_weights[4:], loaded_weights[2:]):
            # für den 2. Convolution-layer passt die shape nicht, bleibt daher unberührt
            if unet_w.shape != loaded_w.shape:
                updated_weights.append(unet_w)

            # alle anderen werden durch die geladenen ersetzt
            else:
                updated_weights.append(loaded_w)

        # Einsetzen der aktualisierten Gewichte
        unet.set_weights(updated_weights)

    return unet

def set_dropout(unet, rgb_drop, ir_drop):
    rgb_names = ['dropout_r', 'dropout_g', 'dropout_b']
    ir_name = 'dropout_ir'

    for layer in unet.layers:
        if layer.name in rgb_names:
            layer.rate = rgb_drop

        if layer.name in ir_name:
            layer.rate = ir_drop


def set_weight_decay(unet, l1, l2):
    regularizer = tf.keras.regularizers.L1L2(l1= l1, l2= l2)

    for layer in unet.layers:
            if isinstance(layer, tf.keras.layers.Conv2D):
                layer.kernel_regularizer = regularizer


def mean_of_RGB_weights(weights):
  # Mittelwert entlang der Kanal-Achse (=-2)
  mean_weights = np.mean(weights, axis=-2).reshape(weights[:,:,-1:,:].shape)
  # Squeeze um Kanalachse = 1 zu kollabieren
  mean_weights = np.squeeze(mean_weights, axis= -2)
  return(mean_weights)
  
def set_pretrained_weights(unet, option):
    unet = unet
    unet_weights = unet.get_weights()

    # Laden der Gewichte des vortrainierten ResNet aus Keras
    RGB_weights_path = './saved_weights/orig_resnet50v2_imagenet_weights.npy'
    saved_weights = np.load(RGB_weights_path, allow_pickle= True)

    # Abschneiden der Classifier-Gewichte
    saved_weights = saved_weights[:-2]


    # Übernehmen der RGB Gewichte für 1. Convolution-layer, IR Gewichte Mittelwert aus RGB
    if option == 'AVG':
        # Gewichte setzen für den Encoder-Teil:
        for i, layer in enumerate(unet_weights):
            # Ende des Encoder-Teils
            if i == len(saved_weights):
                break
            
            # 1. Conv-layer ist i=0
            if i == 0:
                layer[:,:, 3, :] = mean_of_RGB_weights(saved_weights[i])
                layer[:,:, 0:3, :] = saved_weights[i][...]

            # alle anderen Gewichte können übernommen werden
            else:
                layer[...] = saved_weights[i][...]
                
        # Einsetzen der aktualisierten Gewichte
        unet.set_weights(unet_weights)


    # Übernehmen der RGB Gewichte für 1. Convolution-layer, IR Gewichte zufällig
    if option == 'RNDM':
        # Leere Liste für aktualisierte Gewichte
        updated_weights = []

        # Iterieren über geladene Gewichte und zufällige
        for unet_w, loaded_w in zip(unet_weights, saved_weights):
            # Für 1. Conv-Layer stimmt shape nicht überein
            if (unet_w.shape != loaded_w.shape):
                new_weights = unet_w
                # Gewichte für RGB-Channel werden übernommen, IR bleibt wie er ist
                new_weights[:,:, 0:3, :] = loaded_w
                updated_weights.append(new_weights)

            # alle anderen shapes stimmen überein und können übernommen werden
            else:
                updated_weights.append(loaded_w)

        # hinzufügen der zufälligen Gewichte des Decoder-parts
        for unet_w in unet_weights[len(saved_weights):]:
            updated_weights.append(unet_w)

        # Einsetzen der aktualisierten Gewichte
        unet.set_weights(updated_weights)


    # Zusätzlicher Convolution-Layer vor Encoder
    if option == 'RGB':
        # Gewichte setzen für den Encoder-Teil:
        # Beginn ab i=2 durch eingeschobenen Conv-Layer, bis Bottleneck i=269+2
        for i, layer in enumerate(unet_weights):
            if 2 <= i <= 269+2:
                layer[...] = saved_weights[i-2][...]
                
        # Einsetzen der aktualisierten Gewichte
        unet.set_weights(unet_weights)


    # seperate Convolution Layer für RGB und IR
    if option == 'RGB_SPLIT':
        loaded = np.load('./saved_weights/orig_resnet50v2_imagenet_weight_paths.npy', allow_pickle= True)
        loaded = loaded[()]

        # Leere Liste für aktualisierte Gewichte
        updated_weights = []

        # Liste mit Layernamen die später übersprungen werden
        skip_BN = ['conv2_block1_preact_bn.gamma', 'conv2_block1_preact_bn.beta', 'conv2_block1_preact_bn.moving_mean', 'conv2_block1_preact_bn.moving_variance']

        # Iteriere über Layer des Modells
        for l in unet.layers:
            # Falls Gewichte vorhanden für diesen Layer, iteriere über diese   
            if (len(l.weights) > 0):
                for w in l.weights:
                    try:
                        # standardisieren der Layernamen aus layer.weigths und model.get_weigths
                        w_name = w.name.replace('/', '.')[:-2]
                        # durch die beiden Convolutional-Layer verdoppelt sich auch die Anzahl der BN-Gewichte, diese können daher nicht übernommen werden
                        if w_name in skip_BN:
                            updated_weights.append(w)
                            #print(w.name, "not replaced")

                        # für die übrigen Layer werden die Gewichte übernommen, sofern der Layername im Dict vorhanden ist                                    
                        else:
                            updated_weights.append(loaded[w_name])
                            #print(w.name, 'replaced')

                    # ansonsten kommt es zu einer Fehlermeldung und es bleibt es bei den zufälligen Gewichten
                    except KeyError as e:
                        updated_weights.append(w)
                        #print(w.name, "not replaced")

        # Einsetzen der aktualisierten Gewichte
        unet.set_weights(updated_weights)


def set_encoder_frozen(unet, pretrained_weights, train_first_layer= False):
    unet.trainable = True

    # Falls RGB und IR in seperaten Conv-Layern wird RGB immer eingefroren, IR nicht
    if pretrained_weights == 'AVG' or pretrained_weights == 'RNDM':

        for layer in unet.layers:
            # erster Layer des Decoder-parts, ab hier trainierbar
            if layer.name == 'up_sampling2d':
                break
            
            # erster Convolution-Layer trainierbar?
            if train_first_layer and layer.name == 'conv1_conv':
                # "trainable" reguliert ob Gewichte bei back prop angepasst werden
                layer.trainable = True
                continue

            # "trainable" reguliert ob Gewichte bei back prop angepasst werden
            layer.trainable = False        

            # "training" reguliert ob BN Gewichte beim forward pass geändert werden
            if isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.training = False


    if pretrained_weights == 'EXTRA_CONV':

        for layer in unet.layers:
            # erster Layer des Decoder-parts, ab hier trainierbar
            if layer.name == 'up_sampling2d':
                break
            
            # erster Convolution-Layer trainierbar?
            if layer.name == 'conv0_conv':
                # "trainable" reguliert ob Gewichte bei back prop angepasst werden
                layer.trainable = True
                continue

            # "trainable" reguliert ob Gewichte bei back prop angepasst werden
            layer.trainable = False        

            # "training" reguliert ob BN Gewichte beim forward pass geändert werden
            if isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.training = False



    if pretrained_weights == 'RGB_SPLIT':

        for layer in unet.layers:
            # erster Layer des Decoder-parts, ab hier trainierbar
            if layer.name == 'up_sampling2d':
                break
            
            # erster Convolution-Layer trainierbar?
            if layer.name == 'conv1_conv_ir':
                # "trainable" reguliert ob Gewichte bei back prop angepasst werden
                layer.trainable = True
                continue

            # "trainable" reguliert ob Gewichte bei back prop angepasst werden
            layer.trainable = False        

            # "training" reguliert ob BN Gewichte beim forward pass geändert werden
            if isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.training = False

            # beim Split sind für erstes BN keine Gewichte vorhanden, daher trainierbar und Einfrieren danach
            if layer.name == 'conv2_block1_preact_bn':
                layer.training = True
                #freeze_start = True
                continue



def set_trainable_fine_tuning(unet, pretrained_weights, train_first_layer= False):
    # Anzahl der trainierbaren Encoder Layer, durch Versuchsreihe bestimmt
    train_encoder_layers= 27

    # Encoder bleibt größtenteils eingefroren
    set_encoder_frozen(unet, pretrained_weights, train_first_layer)

    # Falls RGB und IR in seperaten Conv-Layern wird RGB immer eingefroren, IR nicht
    #if pretrained_weights in ['AVG', 'RNDM']:

    # Für das Fine-Tuning werden Top_layer des Encoder-Parts wieder trainable geschaltet
    freeze_encoder = False
    countdown = int(train_encoder_layers)
    
    # dafür werden die Layer jetzt rückwärts durchlaufen
    for layer in reversed(unet.layers):
        # ab dem Bottleneck beginnt der Encoder-part
        if layer.name == 'up_sampling2d':
            freeze_encoder = True

        # für train_encoder_layers (int) werden Layer trainierbar
        if freeze_encoder and countdown >= 0:
            # "trainable" reguliert ob Gewichte bei back prop angepasst werden
            layer.trainable = True        

            # "training" reguliert ob BN Gewichte beim forward pass geändert werden
            if isinstance(layer, tf.keras.layers.BatchNormalization):
                layer.training = False

            countdown -= 1


def compile_model(unet, learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate)

    #loss = tf.keras.losses.BinaryFocalCrossentropy(gamma= 2.0, name= 'binary_focal_crossentropy')
    loss = Dice_loss

    binary_iou = tf.keras.metrics.BinaryIoU(name='binary_iou', threshold=0.5),
    metrics = [
        'accuracy',
        binary_iou,
        tf.keras.metrics.TruePositives(name='true_positives'),
        tf.keras.metrics.FalsePositives(name='false_positives'),
        tf.keras.metrics.TrueNegatives(name='true_negatives'),
        tf.keras.metrics.FalseNegatives(name='false_negatives'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

    unet.compile(optimizer= optimizer, loss= loss, metrics= metrics)


def get_callbacks(model_name, output_folder_prefix, do_early_stop):
    checkpoint_path = f'../output/{output_folder_prefix}_checkpoints/{model_name}'
    logger_path = f'../output/{output_folder_prefix}_logger/{model_name}'

    if not os.path.isdir(checkpoint_path):
        os.makedirs(checkpoint_path)

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_binary_iou',
        mode= 'max',
        save_weights_only=False,
        save_best_only=True)

    history_logger = tf.keras.callbacks.CSVLogger(logger_path + '.log')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logger_path, histogram_freq=1)

    callbacks = [checkpoint_callback, history_logger, tensorboard_callback]

    if do_early_stop:
        early_stop =  tf.keras.callbacks.EarlyStopping(
                    monitor='val_binary_iou',
                    min_delta=0,
                    patience=40,
                    verbose=1,
                    mode='max',
                    )

        callbacks.append(early_stop)  

    return callbacks


def combine_log_files(output_folder_prefix, model_name):
    # Zusammenführen der .log Dateien von Initial- und Fine-Tuning Training und Schreiben in neue CSV-Datei
    filenames = [f'../output/{output_folder_prefix}_logger/{model_name}_I.log', f'../output/{output_folder_prefix}_logger/{model_name}.log']
    with open(f'../output/{output_folder_prefix}_logger/{model_name}.csv', 'w') as outfile:
        # spezifizieren des Delimiters für Excel in erster Zeile
        outfile.write('sep=,\n')

        for i, fname in enumerate(filenames):
            with open(fname) as infile:
                reader = csv.reader(infile)

                for j, row in enumerate(reader):
                    # überspringen des 2. Headers
                    if i == 1 and j == 0:
                        continue

                    delimiter = ','
                    list_to_string = delimiter.join(row)
                    list_to_string += '\n'

                    outfile.write(list_to_string)

    

In [8]:
training_split = 0.6
batch_size = 32
patch_size = 224 # Maße des inputs

pretrained_weights = 'AVG' #AVG (Mittelwert von RGB), RNDM (IR-Kanal Random), EXTRA_CONV (Original mit zusätzlichem Conv-Layer davor), RGB_SPLIT (Original und IR Bypass)

conf = {
    'AVG': 'BN',
    'RNDM': 'BN',
    'EXTRA_CONV': 'CONV',
    'RGB_SPLIT': 'SPLIT'
    } # Art des Netzwerks, BN, SPLIT (RGB & IR seperate conv-layer), CONV (zusätzlicher Conv um channel zu downsamplen) ...

learning_rate = 0.001 # Learning rate

rgb_drop = 0.9 # Dropout rate RGB 0-1
ir_drop = 0 # Dropout rate IR 0-1

l1 = 0.0005 # L1 weight decay regularizer 0-1
l2 = 0.0005 # L2 weight decay regularizer0-1

# ob 1. Conv-Layer mit Classifier trainiert wird oder eingefroren während erstem Trainingsdurchlauf des Decoder Parts
train_first_layer = True

# ob 1. Conv-Layer auch während des Fine-Tunings trainiert wird
FT_train_first_layer = True

initial_epochs = 1
fine_tune_epochs = 1
total_epochs = initial_epochs + fine_tune_epochs

early_stop = False

model_name = f'Tensorboard_{pretrained_weights}_rgbDrop_{rgb_drop}_earlyStop_{early_stop}_e{total_epochs}'

# Präfix der checkpoint und logger Ordner im Verzeichnis
output_folder_prefix = 'tensorboard_runs'

unet = unet_4 #load_model(conf[pretrained_weights])
set_dropout(unet, rgb_drop= rgb_drop, ir_drop= ir_drop)
set_weight_decay(unet, l1= l1, l2= l2)
set_pretrained_weights(unet, pretrained_weights)
set_encoder_frozen(unet, pretrained_weights, train_first_layer)
compile_model(unet, learning_rate)

In [9]:
train_data_generator = CustomDataGenerator(
    batch_size = batch_size,
    augment = True,
    shuffle = True,
    img_directory = f'../data/Potsdam/{patch_size}_patches/split_folders{training_split}/train/images',
    msk_directory = f'../data/Potsdam/{patch_size}_patches/split_folders{training_split}/train/masks'
)

val_data_generator = CustomDataGenerator(
    batch_size = batch_size,
    augment = False,
    shuffle = True,
    img_directory = f'../data/Potsdam/{patch_size}_patches/split_folders{training_split}/val/images',
    msk_directory = f'../data/Potsdam/{patch_size}_patches/split_folders{training_split}/val/masks'
)

test_data_generator = CustomDataGenerator(
    batch_size = batch_size,
    augment = False,
    shuffle = False,
    img_directory = f'../data/Potsdam/{patch_size}_patches/split_folders{training_split}/test/images',
    msk_directory = f'../data/Potsdam/{patch_size}_patches/split_folders{training_split}/test/masks'
)

callbacks = get_callbacks(model_name, output_folder_prefix, early_stop)

start = time()

for i, layer in enumerate(unet.layers):
    #if isinstance(layer, tf.keras.layers.BatchNormalization):
    try:
        print(i, layer.name, "trainable weights:", len(layer.trainable_weights), " trainable: " ,layer.trainable, " training: ", layer.training)

    except:
        print(i, layer.name, "trainable weights:", len(layer.trainable_weights), "trainable:", layer.trainable)


model_history = unet.fit(train_data_generator, 
                         validation_data=val_data_generator, 
                         callbacks= callbacks, 
                         epochs= initial_epochs)

# Kopie der ursprünglichen Log, da Fine-tuning-Log sie überschreibt
shutil.copy(f'../output/{output_folder_prefix}_logger/{model_name}.log', f'../output/{output_folder_prefix}_logger/{model_name}_I.log', follow_symlinks=True)

total_epochs =  initial_epochs + fine_tune_epochs

set_trainable_fine_tuning(unet, pretrained_weights, FT_train_first_layer)
# erneut kompilieren, Learning Rate verringern
compile_model(unet, learning_rate/10)


for i, layer in enumerate(unet.layers):
    #if isinstance(layer, tf.keras.layers.BatchNormalization):
    try:
        print(i, layer.name, "trainable weights:", len(layer.trainable_weights), " trainable: " ,layer.trainable, " training: ", layer.training)

    except:
        print(i, layer.name, "trainable weights:", len(layer.trainable_weights), "trainable:", layer.trainable)


history_fine = unet.fit(train_data_generator,
                        validation_data= val_data_generator,
                        callbacks= callbacks,
                        epochs= total_epochs,
                        initial_epoch= initial_epochs)

training_time = time() - start

0 input_resnet50 trainable weights: 0 trainable: False
1 conv1_pad trainable weights: 0 trainable: False
2 conv1_conv trainable weights: 2 trainable: True
3 pool1_pad trainable weights: 0 trainable: False
4 pool1_pool trainable weights: 0 trainable: False
5 conv2_block1_preact_bn trainable weights: 0  trainable:  False  training:  False
6 conv2_block1_preact_relu trainable weights: 0 trainable: False
7 conv2_block1_1_conv trainable weights: 0 trainable: False
8 conv2_block1_1_bn trainable weights: 0  trainable:  False  training:  False
9 conv2_block1_1_relu trainable weights: 0 trainable: False
10 conv2_block1_2_pad trainable weights: 0 trainable: False
11 conv2_block1_2_conv trainable weights: 0 trainable: False
12 conv2_block1_2_bn trainable weights: 0  trainable:  False  training:  False
13 conv2_block1_2_relu trainable weights: 0 trainable: False
14 conv2_block1_0_conv trainable weights: 0 trainable: False
15 conv2_block1_3_conv trainable weights: 0 trainable: False
16 conv2_block1

INFO:tensorflow:Assets written to: ../output/tensorboard_runs_checkpoints\Tensorboard_AVG_rgbDrop_0.9_earlyStop_False_e2\assets


INFO:tensorflow:Assets written to: ../output/tensorboard_runs_checkpoints\Tensorboard_AVG_rgbDrop_0.9_earlyStop_False_e2\assets


199/199 [==============================] - 92s 390ms/step - loss: 0.3773 - accuracy: 0.6308 - binary_iou: 0.4578 - true_positives: 116883688.0000 - false_positives: 103902480.0000 - true_negatives: 84674000.0000 - false_negatives: 14060605.0000 - precision: 0.5294 - recall: 0.8926 - val_loss: 0.3050 - val_accuracy: 0.7243 - val_binary_iou: 0.5674 - val_true_positives: 40389644.0000 - val_false_positives: 24565220.0000 - val_true_negatives: 36370928.0000 - val_false_negatives: 4645926.0000 - val_precision: 0.6218 - val_recall: 0.8968
0 input_resnet50 trainable weights: 0 trainable: False
1 conv1_pad trainable weights: 0 trainable: False
2 conv1_conv trainable weights: 2 trainable: True
3 pool1_pad trainable weights: 0 trainable: False
4 pool1_pool trainable weights: 0 trainable: False
5 conv2_block1_preact_bn trainable weights: 0  trainable:  False  training:  False
6 conv2_block1_preact_relu trainable weights: 0 trainable: False
7 conv2_block1_1_conv trainable weights: 0 trainable: Fal

INFO:tensorflow:Assets written to: ../output/tensorboard_runs_checkpoints\Tensorboard_AVG_rgbDrop_0.9_earlyStop_False_e2\assets


INFO:tensorflow:Assets written to: ../output/tensorboard_runs_checkpoints\Tensorboard_AVG_rgbDrop_0.9_earlyStop_False_e2\assets


199/199 [==============================] - 81s 392ms/step - loss: 0.3048 - accuracy: 0.7290 - binary_iou: 0.5735 - true_positives: 116230416.0000 - false_positives: 71682888.0000 - true_negatives: 116692384.0000 - false_negatives: 14915096.0000 - precision: 0.6185 - recall: 0.8863 - val_loss: 0.2936 - val_accuracy: 0.7324 - val_binary_iou: 0.5776 - val_true_positives: 40054312.0000 - val_false_positives: 23304420.0000 - val_true_negatives: 37558056.0000 - val_false_negatives: 5054918.0000 - val_precision: 0.6322 - val_recall: 0.8879


In [10]:
logger_path = f'../output/{output_folder_prefix}_logger/{model_name}'

In [ ]:
%tensorboard --logger_path logs/fit

In [11]:
unet_4.save('my_model.h5')